In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom

### ASSUMPTIONS:
- Velocity Models
- Receiver Geom

Already made, therefore only need to set the path.

In [2]:
project_dir = '/home/birniece/Documents/Projects/MicroseismicModelling/MicroseismicProcessing/FDModelling/UTAH_Homo_tst'
expname = 'UTAH_Homo_tst'

model_dir = '/home/birniece/Documents/data/UTAH_FORGE/ClairesVMods/homo/UtahFORGE_homo'
rec_file = '/home/birniece/Documents/data/UTAH_FORGE/ClairesVMods/homo/receiver.dat'
source_file = os.path.join('/home/birniece/Documents/data/UTAH_FORGE/SourceFiles','sources.dat')

os.makedirs(project_dir, exist_ok=True)  # make project dir if doesn't already exist

# SELECT PARAMETERS

### MODEL PARAMETERS - Get these from the script that made the models

In [3]:
# TIME PARAMETERS
dt = 0.0001
tdur = 1.0

In [4]:
# SPATIAL PARAMETERS
dx = dy = dz = 10.

xmax = 1700
ymax = 1740 # y is y, I fix the SOFI ordering later
zmax = 1500

nx_homo = int(xmax//dx)
ny_homo = int(ymax//dy)
nz_homo = int(zmax//dz)

n_xzy = [nx_homo, nz_homo, ny_homo]
d_xzy = [dx, dz, dy]

In [5]:
# BOUNDARIES & FREE SURFACE
nbounds = 30
free_surface = True

### SOURCE DEFINITION

In [6]:
strike, dip, rake = 0., 45., 90.

In [7]:
sx, sy, sz = 850, 860, 1300  # y is y, I fix the SOFI ordering later
td = dt  # ignition time
fc = 20  # central frequency
amp = 1.  # amplitude

string_list = [str(i) for i in [sx, sz, sy, td, fc, amp]]

### RECORDING PARAMETERS

In [8]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur/2
snap_step = 10*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 1

In [9]:
# SEISMOGRAM PARAMETERS
smgrm_ndt = 10

### SOFI PARAMETERS

In [10]:
sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8

# WRITE PARAMETERS TO FILE

In [11]:
# SOURCE FILE
text_file = open(source_file, "wt")
text_file.writelines('\t'.join(string_list))
text_file.close()

In [12]:
# FILE NAMES
SOFIjsonfilename = os.path.join(project_dir,'SOFI3D_%s.json'%expname)
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = os.path.join(project_dir,'runSOFI3D_%s.sh'%expname)

In [13]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              expname,
                              moddir=model_dir,
                             )
bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=False)

# SNAP & SEISMOGRAM
monitor_strs = get_monitor_str(tsnap_params, 
                               smgrm_ndt, 
                               expname, 
                               sbsmp_xyz=snap_sbsmp_xyz)


# SOURCE
# src_str = get_source_str(sfile=source_file)
src_str = get_MTsource_str(strike, dip, rake, sfile=source_file)

In [14]:
write_SOFIjsonParams = write_SOFIjsonParams(default_strs, 
                                            monitor_strs,  
                                            t_str, 
                                            mod_strs, 
                                            src_str, 
                                            bndry_str, 
                                            SOFIjsonfilename)
runscript = write_SOFIrunScript(sofimaster, 
                                num_procs, 
                                sofi_param_file, 
                                shellfilename)